In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd

In [2]:
WEBDRIVER_PATH = "D:/Python/webdriver/chromedriver.exe"
URL = "https://www.infomoney.com.br/ferramentas/altas-e-baixas/"

In [3]:
driver = webdriver.Chrome(WEBDRIVER_PATH)
driver.get(URL)

In [4]:
raw_header1 = driver.find_element_by_css_selector("#altas_e_baixas > thead").text.split(" ")
raw_header1

raw_header2 = [
    raw_header1[0], 
    raw_header1[1],
    " ".join(raw_header1[2:4]),     
    " ".join(raw_header1[4:7]),     
    " ".join(raw_header1[7:10]),     
    " ".join(raw_header1[10:13]),     
    " ".join(raw_header1[13:16]),     
    " ".join(raw_header1[16:19]),     
    " ".join(raw_header1[19:21]),     
    " ".join(raw_header1[21:23]),     
    raw_header1[23]     
]
raw_header2[:5]

['ATIVO', 'DATA', 'ÚLTIMO (R$)', 'VAR. DIA (%)', 'VAR. SEM. (%)']

In [5]:
counter = 0
while True:
    counter += 1
    txt = driver.find_element_by_css_selector("#altas_e_baixas > tbody").text.split("\n")
    if txt[0] != 'Carregando...':
        break

    if counter >= 10:
        raise TimeoutError(f"Incapaz de carregar os dados, consulte a disponibilidade de: {URL}")

    sleep(1)

In [6]:
def cleanElement(driver):
    ftxt = []
    for i in txt:
        i = i.split(" ")
        i[-2] = i[-2] + " " + i[-1]
        i.pop()
        ftxt.append(i)

    return ftxt

def add_line(dataFrame, line):
    return dataFrame.append(pd.Series(line, index=dataFrame.columns), ignore_index=True)

In [7]:
df = pd.DataFrame(columns=raw_header2)

for i in cleanElement(txt):
    df = add_line(df, i)


for col in df.iloc[:, 2:-1].columns:
    df[col] = df[col].apply(lambda x: x.replace(",", "."))
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.sort_values(by="VAR. DIA (%)", ascending=False)
df.head()

,ATIVO,DATA,ÚLTIMO (R$),VAR. DIA (%),VAR. SEM. (%),VAR. MÊS (%),VAR. ANO (%),VAR. 12M (%),VAL. MÍN,VAL. MÁX,VOLUME
0,BRDT3,01/07,28.60,7.19,3.70,7.20,38.33,46.39,27.04,29.07,"2,26 B"
1,PRIO3,01/07,20.54,5.38,7.65,5.39,46.32,197.68,19.70,20.55,"554,23 M"
2,RAIL3,01/07,19.75,3.13,1.07,3.13,2.65,-14.02,19.06,19.92,"289,55 M"
3,WEGE3,01/07,34.25,1.66,2.12,1.66,-9.06,31.42,33.47,34.55,"276,72 M"
4,MRFG3,01/07,19.45,1.61,1.89,1.62,35.52,58.97,19.06,19.56,"121,49 M"


In [8]:
df

,ATIVO,DATA,ÚLTIMO (R$),VAR. DIA (%),VAR. SEM. (%),VAR. MÊS (%),VAR. ANO (%),VAR. 12M (%),VAL. MÍN,VAL. MÁX,VOLUME
0,BRDT3,01/07,28.60,7.19,3.70,7.20,38.33,46.39,27.04,29.07,"2,26 B"
1,PRIO3,01/07,20.54,5.38,7.65,5.39,46.32,197.68,19.70,20.55,"554,23 M"
2,RAIL3,01/07,19.75,3.13,1.07,3.13,2.65,-14.02,19.06,19.92,"289,55 M"
3,WEGE3,01/07,34.25,1.66,2.12,1.66,-9.06,31.42,33.47,34.55,"276,72 M"
4,MRFG3,01/07,19.45,1.61,1.89,1.62,35.52,58.97,19.06,19.56,"121,49 M"
...,...,...,...,...,...,...,...,...,...,...,...
72,B3SA3,01/07,16.32,-2.97,-0.94,-2.97,-18.33,-8.42,16.24,16.83,"736,35 M"
73,MULT3,01/07,22.77,-3.10,-5.28,-3.11,-3.23,10.74,22.44,23.56,"232,26 M"
74,TOTS3,01/07,36.46,-3.16,-1.09,-3.16,27.32,55.33,35.85,37.66,"234,91 M"
75,LREN3,01/07,42.83,-3.16,-4.82,-3.17,-1.15,4.47,42.29,44.65,"824,17 M"


In [9]:
df = df.head().append(df.tail())
df.to_excel("DestaquesDoDia.xlsx", index=False)